# <font color=green>1. MEU PRIMEIRO SCRAPING

## 1. Ambiente e bibliotecas

In [1]:
import time
import re
import math

import bs4
import urllib.request as urllib_request
import pandas as pd

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)

BeautifulSoup -> 4.11.1
urllib -> 3.8
pandas -> 1.5.2


In [2]:
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

## 2. Data and Dictionary

In [3]:
def get_keys():
    data = {'Picture Gallery/Hope Board': 0, 'Camera': 1, 'Personality Disorders': 2, 'References/Information': 3, 'App Reports Security Measures in Place': 4, 'Collaborative With Provider/Other': 5, 'Track Mood': 6, 'Geolocation': 7, 'Hybrid': 8, 'Subscription': 9, 'Accessibility': 10, 'Well Written Relevant Content': 11, 'Data Stored on Device': 12, 'Substance Use': 13, 'iCBT or Sleep Therapy': 14, 'For Profit Company': 15, 'DBT': 16, 'Coach/Therapist Connection': 17, 'Eating Disorders': 18, 'Pain': 19, 'Self-Harm': 20, 'Spanish': 21, 'Patient Facing': 22, 'Can Delete Data': 23, 'Sleep': 24, 'Gamification (Points/Badges)': 25, 'Email or Export Your Data': 26, 'Track Symptoms': 27, 'Healthcare Company': 28, 'Graphs of Data': 29, 'Academic Institution': 30, 'Real Time Response': 31, 'User Generated Data': 32, 'Does What it Claims': 33, 'Payment': 34, 'Deep Breathing': 35, 'External Devices': 36, 'Is PHI Shared': 37, 'Claims to Not Replace Care': 38, 'Notifications': 39, 'Reminders': 40, 'Meets HIPAA': 41, 'CBT': 42, 'Send Your Data to a Medical Record': 43, 'Assessments/Screenings': 44, 'Has Crisis Management Feature': 45, 'Peer Support': 46, 'Self Help': 47, 'App Declares Data Use and Purpose': 48, 'AI Support': 49, 'In-App Purchase': 50, 'Summary of Data': 51, 'Psychoeducation': 52, 'Social Network': 53, 'Substance Use (Alcohol)': 54, 'Mood Disorders': 55, 'Reference': 56, 'Substance Use (Smoking & Tobacco)': 57, 'Free to Download': 58, 'Mindfulness': 59, 'Non-Profit Company': 60, 'Supporting Studies': 61, 'Data Stored on Server': 62, 'Offline': 63, 'PTSD': 64, 'Schizophrenia': 65, 'Videos': 66, 'OCD': 67, 'Physical Health Exercises': 68, 'Productivity': 69, 'Biodata': 70, 'Link to Formal Care/Coaching': 71, 'Phobias': 72, 'De-Identified/Anonymized Data Shared': 73, 'Government': 74, 'Headache': 75, 'Diary': 76, 'Track Medication': 77, 'Chat/Message': 78, 'Track Sleep': 79, 'Microphone': 80, 'Network Support': 81, 'Contact List': 82, 'Bipolar Disorder': 83, 'Stress & Anxiety': 84, 'Surveys': 85, 'Goal Setting/Habits': 86, 'Bio Feedback with Sense Data': 87, 'ACT': 88, 'Can Cause Harm': 89, 'Aggregated Data Shared': 90, 'Has Privacy Policy': 91, 'Chatbot Interaction': 92, 'Can Opt Out of Data Collection': 93, 'Physical Health': 94, 'Totally Free': 95, 'Journaling': 96, 'Asynchronous Response': 97, 'Own Your Own Data': 98, 'Audio/Music/Scripts': 99, 'Non-Specific': 100, 'Appropriately Advises Patient in Case of Emergency': 101, 'Step Count': 102, 'Identify New Conditions': 103, 'Cardiovascular Health': 104}
    return data

In [4]:
def get_dataframe(data):
    keys = ['Name'] + [key for key in data]
    df = pd.DataFrame(columns=keys)
    
    return df

In [5]:
cost_opt = set()
privacy_opt = set()
foundation_opt = set()
features_opt = set()
conditions_opt = set()
engagements_opt = set()
inputs_opt = set()
outputs_opt = set()
uses_opt = set()
dev_types_opt = set()
access_opt = set()

In [44]:
# Create categories dictionary
categories = set()

for index in cost_opt:
    categories.add(index)
for index in privacy_opt:
    categories.add(index)
for index in foundation_opt:
    categories.add(index)
for index in features_opt:
    categories.add(index)
for index in conditions_opt:
    categories.add(index)
for index in engagements_opt:
    categories.add(index)
for index in inputs_opt:
    categories.add(index)
for index in outputs_opt:
    categories.add(index)
for index in uses_opt:
    categories.add(index)
for index in dev_types_opt:
    categories.add(index)
for index in access_opt:
    categories.add(index)
    
count = 0
dictionary = dict()
for index in categories:
    dictionary[index] = count
    count += 1

print(dictionary)

{'Appropriately Advises Patient in Case of Emergency': 0, 'Chatbot Interaction': 1, 'Well Written Relevant Content': 2, 'AI Support': 3, 'Email or Export Your Data': 4, 'Supporting Studies': 5, 'Healthcare Company': 6, 'Cardiovascular Health': 7, 'Spanish': 8, 'Payment': 9, 'Physical Health Exercises': 10, 'Substance Use (Alcohol)': 11, 'Does What it Claims': 12, 'Audio/Music/Scripts': 13, 'Social Network': 14, 'Step Count': 15, 'Notifications': 16, 'Phobias': 17, 'Send Your Data to a Medical Record': 18, 'Chat/Message': 19, 'Diary': 20, 'Can Opt Out of Data Collection': 21, 'App Reports Security Measures in Place': 22, 'Identify New Conditions': 23, 'Is PHI Shared': 24, 'Videos': 25, 'Mindfulness': 26, 'Self Help': 27, 'Summary of Data': 28, 'Geolocation': 29, 'Asynchronous Response': 30, 'Psychoeducation': 31, 'Contact List': 32, 'Graphs of Data': 33, 'App Declares Data Use and Purpose': 34, 'Totally Free': 35, 'Reference': 36, 'Biodata': 37, 'Stress & Anxiety': 38, 'Link to Formal C

## 3. Functions

In [6]:
def str_to_valid_str(str_data):
    return " ".join(str_data.split()).replace("> <", "><")

In [7]:
def bytes_to_valid_str(b_data):
    str_data = b_data.decode("utf-8")
    return str_to_valid_str(str_data)

In [8]:
def get_html(page):
    html = str_to_valid_str(page)
    
    soup = BeautifulSoup(html, "html.parser")
    return soup.prettify()

In [9]:
def click_button(driver, btn):
    driver.execute_script("arguments[0].click();", btn)

In [10]:
def close_popup(driver):
    try:
        close_btn = None
        btns = driver.find_elements(By.TAG_NAME, 'button')
        for btn in btns:
            if btn.text == "Close":
                print("    Found button")
                close_btn = btn
        if close_btn != None:
            click_button(driver, close_btn)

    except StaleElementReferenceException as e:
        print("\n\n\n    Popup closed ", e.reason)

In [11]:
def change_to_table_view(driver):
    try:
        close_btn = None
        btns = driver.find_elements(By.TAG_NAME, 'button')
        for btn in btns:
            if btn.text == "Table View":
                close_btn = btn
        click_button(driver, close_btn)

    except StaleElementReferenceException as e:
        print("\n\n\n    Change to table view ", e.reason)

In [12]:
def scroll_page(driver, element, value):
    driver.execute_script("arguments[0].scroll(0," + str(value) + ");", element)

In [13]:
def get_from_xpath(driver, path):
    return driver.find_element(By.XPATH, path)

In [14]:
def get_info(driver):
    cost = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[3]')
    privacy = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[4]')
    foundation = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[5]')
    features = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[6]')
    conditions = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[7]')
    engagements = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[8]')
    inputs = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[9]')
    outputs = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[10]')
    uses = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[11]')
    dev_types = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[12]')
    access = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[13]')
    
    return (cost, privacy, foundation, features, conditions, engagements, inputs, outputs, uses, dev_types, access)

In [15]:
def mount_tag(cost, privacy, foundation, features, conditions, engagements, inputs, outputs, uses, dev_types, access):
    tags = re.split('\n', cost.text)[1:]
    tags = tags + re.split('\n', privacy.text)[1:]
    tags = tags + re.split('\n', foundation.text)[1:]
    tags = tags + re.split('\n', features.text)[1:]
    tags = tags + re.split('\n', conditions.text)[1:]
    tags = tags + re.split('\n', engagements.text)[1:]
    tags = tags + re.split('\n', inputs.text)[1:]
    tags = tags + re.split('\n', outputs.text)[1:]
    tags = tags + re.split('\n', uses.text)[1:]
    tags = tags + re.split('\n', dev_types.text)[1:]
    tags = tags + re.split('\n', access.text)[1:]
    
    return tags

In [16]:
def update_dict(cost, privacy, foundation, features, conditions, engagements, inputs, outputs, uses, dev_types, access):
    for item in re.split('\n', cost.text)[1:]:
        cost_opt.add(item)
    for item in re.split('\n', privacy.text)[1:]:
        privacy_opt.add(item)
    for item in re.split('\n', foundation.text)[1:]:
        foundation_opt.add(item)
    for item in re.split('\n', features.text)[1:]:
        features_opt.add(item)
    for item in re.split('\n', conditions.text)[1:]:
        conditions_opt.add(item)
    for item in re.split('\n', engagements.text)[1:]:
        engagements_opt.add(item)
    for item in re.split('\n', inputs.text)[1:]:
        inputs_opt.add(item)
    for item in re.split('\n', outputs.text)[1:]:
        outputs_opt.add(item)
    for item in re.split('\n', uses.text)[1:]:
        uses_opt.add(item)
    for item in re.split('\n', dev_types.text)[1:]:
        dev_types_opt.add(item)
    for item in re.split('\n', access.text)[1:]:
        access_opt.add(item)

In [41]:
def get_data(driver, df, data):
    column = list(df.columns.values)
    time.sleep(5) # Load page
    
    # Get info about number of comments
    coments_info = driver.find_element(
        By.XPATH, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[4]'
    )
    time.sleep(10) # Load info
    quant = int(re.split('\)|\(', coments_info.text)[1]) # Get coments quantity
    
    # Get header data
    name = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[2]/div/div[2]/div/div/div/div[1]/h5')
    supported_study = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[2]/div/div[3]/div/div[3]/div/div[2]/div/div[2]/span')
    
    coments_quant[name.text] = quant
    print("name: ", name.text, " has study: ", supported_study.text, " comentários: ", quant)
    # Get each comment    
    for i in range(quant):
        print('get data')
        # Get data
        cost = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[3]')
        privacy = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[4]')
        foundation = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[5]')
        features = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[6]')
        conditions = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[7]')
        engagements = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[8]')
        inputs = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[9]')
        outputs = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[10]')
        uses = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[11]')
        dev_types = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[12]')
        access = get_from_xpath(driver, '//*[@id="app-content"]/div[2]/div/div[4]/div/div[5]/div/div/div[13]')
        
        # Get tags
        tags = mount_tag(cost, privacy, foundation, features, conditions, engagements, inputs, outputs, uses, dev_types, access)
        
        # Assemble line
        base_line = [name.text] + ([0] * (len(column) - 1))
        print("base_line X dictionary: ", len(base_line), " = ", len(column))
        for index in tags:
            if index in data:
                base_line[data[index] + 1] = 1
            else:
                print("NÃO ENCONTRADO ", index)
                # Add new tag to dict
                # Add new column to the df
                # add new column to the new line
        
        # Add line to dataframe
        new_line = pd.DataFrame([base_line], columns=column)
        df = pd.concat([df, new_line], ignore_index = True)
        
        # Update tags database
        update_dict(cost, privacy, foundation, features, conditions, engagements, inputs, outputs, uses, dev_types, access)
        
        # Button to the next comment
        if i < quant - 1:
            next_btn = driver.find_element(By.CSS_SELECTOR,
                '#app-content > div.jss5.jss8 > div > div:nth-child(4) > div > div:nth-child(6) > div > nav > ul > li:last-child > button'
            )
            click_button(driver, next_btn) # Go to comment
            time.sleep(10) # Load info
    
    return df

## 4. Web scrapping com Selenium 

In [42]:
def selenium_scrapping(start, end):
    options = Options()
    # options.headless = True
    options.add_argument("start-maximized");
    
    dictionary = get_keys()
    df = get_dataframe(dictionary)

    DRIVER_PATH = '/home/pizzaisamor/chromedriver/chromedriver'
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

    driver.get('https://mindapps.org/Apps')
    
    try:
        element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, "app-content"))
        )
        
        # close_popup(driver)
        time.sleep(6) # Load page
        
        for line in range(0, start, 4):
            apps_list = driver.find_element(
                By.XPATH, '//*[@id="app-content"]/div[2]/div[3]/div[3]/div/div/div[1]'
            )
            line = math.floor(line / 4) # Find right line 
            scroll_page(driver, apps_list, line * 536) # Go to the line
            time.sleep(3) # Wait to update DOM
    
        for card in range(start, end):
            print("\nCard: ", card)
        
            # Scroll apps list
            apps_list = driver.find_element(
                By.XPATH, '//*[@id="app-content"]/div[2]/div[3]/div[3]/div/div/div[1]'
            )
            line = math.floor(card / 4) # Find right line 
            scroll_page(driver, apps_list, line * 536) # Go to the line
            time.sleep(3) # Wait to update DOM
        
            # Find card
            navigate = driver.find_element(
                By.XPATH, '//div[@data-index='+ str(card) +']/div'
            )
    
            click_button(driver, navigate) # Click on the card
            df = get_data(driver, df, dictionary) # Get the data from the page
            driver.back() # Go to main menu
            time.sleep(8) # Load page
        
        display(df)
    
    finally:
        driver.quit()
        return df


In [43]:
coments_quant = dict()
dataset = get_dataframe(get_keys())
count = 0
seq_size = 20

# Start in 0 end in 612
for seq in range(0, 610, seq_size):
    last_value = min([(seq + seq_size), 610])
    
    print("Intervalo: ", seq, " - ", last_value)
    df = selenium_scrapping(seq, seq + seq_size)
    
    dataset = pd.concat([dataset, df], ignore_index = True)
    
    file_name = 'web-scrapping_v' + str(count) + '.csv'
    dataset.to_csv(file_name, encoding='utf-8', index=False)
    count += 1

Intervalo:  0  -  20


/tmp/ipykernel_81919/758690503.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)



Card:  0
name:  Intellect: Create a Better You  has study:  No  comentários:  1
get data
base_line X dictionary:  106  =  106

Card:  1
name:  Bipolar Test  has study:  No  comentários:  9
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106

Card:  2
Intervalo:  20  -  40


/tmp/ipykernel_81919/758690503.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)



Card:  20
name:  MEDITATION BY MT – SELF HELP, STRESS RELIEF  has study:  No  comentários:  6
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106

Card:  21
name:  TalkLife  has study:  No  comentários:  9
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106

Card:  22
name:  Chronic Pain Tracker Lite  has study:  No  comentários:  5
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:

...


Card:  580
name:  ChatPal - A Psychology ChatBot  has study:  Yes  comentários:  2
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106

Card:  581
name:  My Spiritual Toolkit AA Steps  has study:  No  comentários:  3
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106

Card:  582
name:  Flamingo - Mood Tracker, Diary  has study:  No  comentários:  2
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106

Card:  583
name:  Counter Tally Count  has study:  No  comentários:  5
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106

Card:  584
name:  Less - Alcohol Tracker  has study:  No  comentários:  3
get data
base_line X dictionary:  106  =  106
get da

,Name,Picture Gallery/Hope Board,Camera,Personality Disorders,References/Information,App Reports Security Measures in Place,Collaborative With Provider/Other,Track Mood,Geolocation,Hybrid,...,Totally Free,Journaling,Asynchronous Response,Own Your Own Data,Audio/Music/Scripts,Non-Specific,Appropriately Advises Patient in Case of Emergency,Step Count,Identify New Conditions,Cardiovascular Health
0,ChatPal - A Psychology ChatBot,0,0,0,0,1,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,ChatPal - A Psychology ChatBot,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
2,My Spiritual Toolkit AA Steps,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,My Spiritual Toolkit AA Steps,0,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
4,My Spiritual Toolkit AA Steps,0,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,"Sweet Sleep : Relax, Meditatio",0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
59,"Sweet Sleep : Relax, Meditatio",0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
60,Quit smoking with Quitify PRO,0,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
61,Quit smoking with Quitify PRO,0,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


Intervalo:  600  -  610


/tmp/ipykernel_81919/758690503.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)



Card:  600
name:  Alcohol Tracker - I Quit Drink  has study:  No  comentários:  3
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106

Card:  601
name:  The Alcohol Experiment  has study:  No  comentários:  2
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106

Card:  602
name:  Quit Smoking - Stop Smoking Now  has study:  No  comentários:  3
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106

Card:  603
name:  Sober Sidekick: Quit Drinking  has study:  No  comentários:  4
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106
get data
base_line X dictionary:  106  =  106

Card:  604
name:  CBT Self-Care Journal  has study:  No  comentários:  1
get data
base_line X dictionary:  106  =  106

Car

In [45]:
cost_opt

{'Free to Download',
 'In-App Purchase',
 'Payment',
 'Subscription',
 'Totally Free'}

In [46]:
privacy_opt

{'Aggregated Data Shared',
 'App Declares Data Use and Purpose',
 'App Reports Security Measures in Place',
 'Can Delete Data',
 'Can Opt Out of Data Collection',
 'Data Stored on Device',
 'Data Stored on Server',
 'De-Identified/Anonymized Data Shared',
 'Has Privacy Policy',
 'Is PHI Shared',
 'Meets HIPAA'}

In [47]:
foundation_opt

{'Appropriately Advises Patient in Case of Emergency',
 'Can Cause Harm',
 'Claims to Not Replace Care',
 'Does What it Claims',
 'Patient Facing',
 'Supporting Studies',
 'Well Written Relevant Content'}

In [48]:
features_opt

{'ACT',
 'Bio Feedback with Sense Data',
 'Biodata',
 'CBT',
 'Chatbot Interaction',
 'Coach/Therapist Connection',
 'DBT',
 'Deep Breathing',
 'Goal Setting/Habits',
 'Identify New Conditions',
 'Journaling',
 'Mindfulness',
 'Peer Support',
 'Physical Health',
 'Physical Health Exercises',
 'Picture Gallery/Hope Board',
 'Productivity',
 'Psychoeducation',
 'Track Medication',
 'Track Mood',
 'Track Sleep',
 'Track Symptoms',
 'iCBT or Sleep Therapy'}

In [49]:
conditions_opt

{'Bipolar Disorder',
 'Cardiovascular Health',
 'Eating Disorders',
 'Headache',
 'Mood Disorders',
 'Non-Specific',
 'OCD',
 'PTSD',
 'Pain',
 'Personality Disorders',
 'Phobias',
 'Schizophrenia',
 'Self-Harm',
 'Sleep',
 'Stress & Anxiety',
 'Substance Use',
 'Substance Use (Alcohol)',
 'Substance Use (Smoking & Tobacco)'}

In [50]:
engagements_opt

{'AI Support',
 'Assessments/Screenings',
 'Asynchronous Response',
 'Audio/Music/Scripts',
 'Chat/Message',
 'Collaborative With Provider/Other',
 'Gamification (Points/Badges)',
 'Network Support',
 'Peer Support',
 'Real Time Response',
 'User Generated Data',
 'Videos'}

In [51]:
inputs_opt

{'Camera',
 'Contact List',
 'Diary',
 'External Devices',
 'Geolocation',
 'Microphone',
 'Social Network',
 'Step Count',
 'Surveys'}

In [52]:
outputs_opt

{'Graphs of Data',
 'Link to Formal Care/Coaching',
 'Notifications',
 'References/Information',
 'Reminders',
 'Social Network',
 'Summary of Data'}

In [53]:
uses_opt

{'Hybrid', 'Reference', 'Self Help'}

In [54]:
dev_types_opt

{'Academic Institution',
 'For Profit Company',
 'Government',
 'Healthcare Company',
 'Non-Profit Company'}

In [55]:
access_opt

{'Accessibility',
 'Email or Export Your Data',
 'Offline',
 'Own Your Own Data',
 'Send Your Data to a Medical Record',
 'Spanish'}

In [56]:
coments_quant

{'Intellect: Create a Better You': 1,
 'Bipolar Test': 4,
 'MEDITATION BY MT – SELF HELP, STRESS RELIEF': 6,
 'TalkLife': 9,
 'Chronic Pain Tracker Lite': 5,
 'TF-CBT Triangle of Life': 7,
 'AbleTo': 8,
 'CBT-i Coach': 10,
 'UCLA Mindful': 7,
 'Deep Sleep, Insomnia Help': 7,
 'minding - meditation timer': 9,
 'DipQuit Pro: Quit Dipping Smok': 6,
 'iMoodJournal': 7,
 'HearMe | Empathy Not Therapy': 8,
 'Smiling Mind': 9,
 'Action for Happiness: Find tip': 8,
 'Simple Habit: Meditation': 7,
 'Smoke Out: Tobacco Pirates': 4,
 'Migraine Relief Hypnosis - Hea': 5,
 'No Relapse: overcome addiction': 5,
 'Worry Watch: Anxiety & Mood': 6,
 'HERE - Interactive Meditation': 11,
 'MentalFlow - Your Best Self.': 8,
 'Moodflow: Mood Tracker': 7,
 'Menda: Chronic Pain Relief': 4,
 'I Am Sober': 10,
 'Quit Smoking - Premium Subliminal Messages': 4,
 'PIER': 8,
 'Mood Care Journal－Self Tracker': 6,
 'Lift – Depression & Anxiety': 7,
 'Schizophrenia': 2,
 'Sleep Cycle: Sleep Tracker': 5,
 'Moodnotes - 